# EE 467 Lab 7: Instruction Set Architecture (ISA) Identification of Program Binaries

Welcome to Lab 7 of EE 467! Today we apply the machine learning techniques you’ve learned throughout the course to a practical cybersecurity task: Instruction Set Architecture (ISA) detection. We will use a dataset with **50k Base64-encoded binaries** downloaded from Praetorian’s “Machine Learning Binaries" challenge web page [1]. Each of the encoded binary string in this dataset consists of 88 characters (66 bytes) on average and belongs to one of the following **twelve architecture types: avr, alphaev56, arm, m68k, mips, mipsel, powerpc, s390, sh4, sparc, x86_64, and xtensa**.

We will use two different feature extration models, **byte-histogram+endianness features and byte-level TF-IDF features**, to extract features from the binaries. Using these features, we will **train and test SVM, Logisitic Regression, Decision Tree, and Random Forrest** classification algorithms. As in the previous labs, all algorithms are evaluated by **accuracy, precision, recall and F1-score**.

## Preparation

Like previous labs, we start by installing all dependencies needed for this lab:

In [1]:
%pip install numpy scipy scikit-learn termcolor

  Using cached termcolor-3.3.0-py3-none-any.whl.metadata (6.5 kB)
Using cached termcolor-3.3.0-py3-none-any.whl (7.7 kB)
Note: you may need to restart the kernel to use updated packages.


## Data Pre-processing

We load the raw binary data and their labels from separate files. During loading we decode each Base64 string into bytes, preview a handful of samples in three representations (Base64, hex, and byte-level), deduplicate entries, and compute summary statistics over the full dataset:


In [2]:
!tar -xJf binaries-dataset.tar.xz #<---- For unzipping the .zip dataset folder

In [4]:
import base64
from collections import Counter

from termcolor import colored

# Path of binaries and labels files
BINARIES_PATH = "./binaries-dataset/Base64EcncodedBinaries-50k.txt"
LABELS_PATH = "./binaries-dataset/LabelsOfBinaries-50k.txt"
# Number of samples to display
N_DISPLAY_SAMPLES = 5

# Binaries and labels
binaries = []
binaries_set = set()
raw_labels = []
# Duplicated samples and displayed samples counts
dup_count = 0
display_count = 0

# Decode each Base64-encoded binary into raw bytes, display a few samples in multiple
# representations, and deduplicate entries before storing them.
# Note: Each byte (8 bits, 256 possible values) is represented by 2 hex characters.
with open(BINARIES_PATH) as binaries_file, open(LABELS_PATH) as binaries_label_file:
    for line_tmp, label_tmp in zip(binaries_file, binaries_label_file):
        # Remove EOL characters
        line_eol_removed = line_tmp.rstrip()
        raw_label = label_tmp.rstrip()

        # [ TODO ]
        # 1. Decode Base64-encoded binaries into byte strings
        binary_sample = base64.b64decode(line_eol_removed) 

        # Display the first N_DISPLAY_SAMPLES binaries in three representations
        if display_count < N_DISPLAY_SAMPLES:
            # [ TODO ]
            # 2. Encode byte strings as hex strings
            hex_encoded = binary_sample.hex()
            # 3. Rewrite encoded hex strings in byte-level granularities
            #    (i.e. Separate byte data by spaces)
            byte_level = " ".join([hex_encoded[i:i+2] for i in range(0, len(hex_encoded), 2)])

            print(colored("Base64-encoded string:", attrs=["bold"]))
            print(colored(line_eol_removed, "red"))

            print(colored("Hex-encoded string:", attrs=["bold"]))
            print(colored(hex_encoded, "blue"))

            print(colored("Byte level decomposition:", attrs=["bold"]))
            print(colored(byte_level, "green"))
            print()

            display_count += 1

        # Skip duplicates; otherwise add sample to the dataset
        if binary_sample not in binaries_set:
            binaries.append(binary_sample)
            binaries_set.add(binary_sample)
            raw_labels.append(raw_label)
        else:
            dup_count += 1

# Count labels
labels_info = Counter(raw_labels)
min_sample_size = min(labels_info.values())

# Compute and print statistics
print(colored("[[ Dataset Statistics ]]", attrs=["bold"]))

print("* Distinct labels:", labels_info.keys())
print("* # of samples for all classes:", labels_info)
print("* Min # of samples in class:", min(labels_info.values()))
print("* # of samples in total:", sum(labels_info.values()))
print("* # of duplicates:", dup_count)

Base64-encoded string:
AC0GLQYCdQtKBuawYVrRiiFa0iYx/+AKYANm+XZR0iqEDGMrhAxhGGE7ISyEDGVTYChACyEthChAGEAbIANlag==
Hex-encoded string:
002d062d0602750b4a06e6b0615ad18a215ad22631ffe00a600366f97651d22a840c632b840c6118613b212c840c65536028400b212d84284018401b2003656a
Byte level decomposition:
00 2d 06 2d 06 02 75 0b 4a 06 e6 b0 61 5a d1 8a 21 5a d2 26 31 ff e0 0a 60 03 66 f9 76 51 d2 2a 84 0c 63 2b 84 0c 61 18 61 3b 21 2c 84 0c 65 53 60 28 40 0b 21 2d 84 28 40 18 40 1b 20 03 65 6a

Base64-encoded string:
AAAgkwAAkJMAAICTAACCgZOBkJMAAICTAACEgZWBkJMAAICTAACAkQAAkOCQkwAAgJMAAB+SH5Ifkh+SH5Ifkg==
Hex-encoded string:
00002093000090930000809300008281938190930000809300008481958190930000809300008091000090e090930000809300001f921f921f921f921f921f92
Byte level decomposition:
00 00 20 93 00 00 90 93 00 00 80 93 00 00 82 81 93 81 90 93 00 00 80 93 00 00 84 81 95 81 90 93 00 00 80 93 00 00 80 91 00 00 90 e0 90 93 00 00 80 93 00 00 1f 92 1f 92 1f 92 1f 92 1f 92 1f 92

Base64-encoded string:
AAAAAAAAAADAIAAoA

The statistics indicate that our dataset is largely balanced — all twelve ISA classes have similar sample counts, so no undersampling or oversampling is needed. We only need to map the string labels to integer indices for use with scikit-learn:


In [5]:
import numpy as np

# Map each unique string label to a consecutive integer (0, 1, 2, ...)
raw_labels_to_labels = {label: i for i, label in enumerate(labels_info.keys())}

# Build the integer label array used by all scikit-learn models below
labels = np.array([raw_labels_to_labels[raw_label] for raw_label in raw_labels])

## Feature Extraction

In this lab we are going to try three different feature extraction techniques:

* Byte-Histogram and Endianness Features
* Byte-level 1,2,3-Gram TF-IDF Features
* Hex-level (4-bit) 1,2,3-Gram TF-IDF Features

### Byte-Histogram and Endianness Features

This feature extraction method was originally proposed in [2]. It builds two histograms from raw binary data:

1. **Byte-value histogram**: Scans the binary one byte at a time, producing a 256-entry histogram (one bin per possible byte value, 0–255).
2. **Endianness histogram**: Scans for four specific two-byte (word) patterns — `00 01`, `01 00`, `ff fe`, and `fe ff` — whose frequency hints at the endianness of the binary data.

Both histograms are concatenated and then **normalized** by the total number of bytes in the binary, yielding a 260-dimensional feature vector that is comparable across binaries of different lengths.

In [6]:
from collections.abc import Iterator

# The four two-byte patterns whose presence signals endianness
ENDIANNESS_WORDS = (b"\x00\x01", b"\x01\x00", b"\xff\xfe", b"\xfe\xff")

def pairwise(bstr: bytes) -> Iterator[bytes]:
    """
    Yield every consecutive overlapping byte pair in `bstr`.

    Example: b"uvwxyz" -> b"uv", b"vw", b"wx", b"xy", b"yz"
    """
    bstr_iter = iter(bstr)
    char_a = next(bstr_iter)

    for char_b in bstr_iter:
        yield char_a + char_b
        char_a = char_b

def make_byte_hist_endian_feature(binary: bytes) -> np.ndarray:
    """
    Return a 260-dim normalized feature vector for a single binary.

    The vector is the concatenation of:
      - a 256-entry byte-value histogram
      - a 4-entry endianness-word histogram
    divided element-wise by the binary's byte length.
    """
    # Count occurrences of each byte value (0–255)
    byte_hist = np.zeros(256, dtype=int)
    for byte_data in binary:
        byte_hist[byte_data] += 1

    # Count occurrences of the four endianness-indicating word patterns
    word_hist = np.zeros(len(ENDIANNESS_WORDS), dtype=int)
    for word_data in pairwise(binary):
        try:
            word_idx = ENDIANNESS_WORDS.index(word_data)
            word_hist[word_idx] += 1
        except ValueError:
            # Not an endianness marker; skip
            pass

    # Concatenate and normalize by binary length so features are scale-invariant
    concat_hist = np.concatenate((byte_hist, word_hist))
    return concat_hist / len(binary)

feats_byte_hist_endian = np.stack([make_byte_hist_endian_feature(binary) for binary in binaries])

### Byte-level 1,2,3-Gram TF-IDF Features

This method generalizes the byte-histogram approach using TF-IDF weighting over byte n-grams (unigrams, bigrams, and trigrams). A few implementation notes:

* We reuse scikit-learn's `TfidfVectorizer` with the character (`"char"`) analyzer so it treats each byte as a single character. Setting the encoding to `latin1` ensures every possible byte value (0–255) maps to a valid character without errors.
* For trigrams, we cap the vocabulary with `max_features` to keep feature dimensionality (and training time) manageable.
* Unigram+bigram and trigram matrices are merged in the feature dimension using `scipy.sparse.hstack`, matching the approach from homework 1.


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

# Limit trigram vocabulary to keep memory and training time manageable
TF_IDF_3_MAX_FEATS = 10000

# [ TODO ]
# Extract Byte-level (1,2,3)-gram TF-IDF features using TfidfVectorizer

# 1. Fit character (1,2)-gram TF-IDF model using `binaries` as training data, with `encoding` set to
#    "latin1". Save transformed features as `feats_tf_idf_12`.
tf_idf_12 = TfidfVectorizer(
    analyzer="char",
    ngram_range=(1, 2),
    encoding="latin1"
)
feats_tf_idf_12 = tf_idf_12.fit_transform(binaries)

# 2. Fit character trigram TF-IDF model using `binaries` as training data, with `encoding` set to
#    "latin1" and number of features limited by `TF_IDF_3_MAX_FEATS`. Save transformed features as
#    `feats_tf_idf_3`.
tf_idf_3 = TfidfVectorizer(
    analyzer="char",
    ngram_range=(3, 3),
    encoding="latin1",
    max_features=TF_IDF_3_MAX_FEATS
)
feats_tf_idf_3 = tf_idf_3.fit_transform(binaries)

# 3. Concatenate `feats_tf_idf_12` and `feats_tf_idf_3` into `feats_tf_idf_123` using `scipy.sparse.hstack`.
feats_tf_idf_123 = hstack([feats_tf_idf_12, feats_tf_idf_3])

### Hex-level (4-bit) 1,2,3-Gram TF-IDF Features

This method applies the same TF-IDF n-gram approach but operates on the **hex-encoded** representation of each binary. Because each hex character encodes four bits (half a byte), the resulting vocabulary captures finer-grained patterns than the byte-level approach, at the cost of doubling the sequence length. No `latin1` encoding override is needed here since all hex characters are standard ASCII.


In [20]:
# [ TODO ]
# Extract Hex-level (1,2,3)-gram TF-IDF features using TfidfVectorizer
# 1. Hex-encode all binary data in `binaries` and save them as `binaries_hex`
binaries_hex = [binary.hex() for binary in binaries]
# 2. Fit character (1,2,3)-gram TF-IDF model using `binaries_hex` as training data. Save transformed features
#    as `feats_tf_idf_hex_123`.


feats_tf_idf_hex_123 = TfidfVectorizer(
    analyzer="char",
    ngram_range=(1, 3),
).fit_transform(binaries_hex)


## Training and Testing Different ML Models

Now that we have multiple feature representations, we evaluate each one across four classifiers: **Linear SVM**, **Logistic Regression**, **Decision Tree**, and **Random Forest**. To keep runtimes reasonable with a 50k-sample dataset, we use a stratified 20%/5% train/test split. We report accuracy, precision, recall, and F1-score, as well as the confusion matrix, for both the training and test sets.

We start by implementing a shared `train_test_ml_models` function that encapsulates the complete train-evaluate loop so we can reuse it for each feature type:


In [14]:
import time
from contextlib import contextmanager

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Default random seed for reproducibility
RNG_SEED = 42

@contextmanager
def timeit(action: str):
    """Context manager that prints elapsed wall-clock time for a code block."""
    start_time = time.time()
    print(f"Timing started for {action} ...")

    yield
    elapsed_time = time.time() - start_time
    print(f"Timing ended for {action}. Elapsed time: {elapsed_time:.2f}s")

def train_test_ml_models(feats: np.ndarray, labels: np.ndarray, train_size: float = 0.2, test_size: float = 0.05,
    rng_seed_data: int = RNG_SEED, rng_seed: int = RNG_SEED):
    """
    Train and evaluate four classifiers on the given features and labels.

    Uses a stratified train/test split to preserve class balance, then reports
    confusion matrices and full classification reports for both subsets.
    """
    # Stratified split keeps class proportions consistent across train and test sets
    binaries_train, binaries_test, labels_train, labels_test = train_test_split(
        feats, labels, train_size=train_size, test_size=test_size, stratify=labels, random_state=rng_seed_data
    )

    # List of ML models to train and evaluate
    ml_models = {
        "SVM": LinearSVC(max_iter=200, random_state=rng_seed),
        "Logistic Regression": LogisticRegression(max_iter=200, random_state=rng_seed),
        "Decision Tree": DecisionTreeClassifier(random_state=rng_seed),
        "Random Forest": RandomForestClassifier(random_state=rng_seed)
    }

    for model_name, model in ml_models.items():
        print(colored(f"[[ Training and Evaluation for {model_name} ]]", attrs=["bold"]))
        print(colored("[ Training ]", attrs=["bold"]))

        with timeit(action=f"training of {model_name}"):
            model.fit(binaries_train, labels_train)

        # Compute and show metrics for training and testing sets
        for subset_name, binaries_eval, labels_eval in (
            ("Training", binaries_train, labels_train),
            ("Testing", binaries_test, labels_test)
        ):
            preds = model.predict(binaries_eval)

            print(colored(f"[ {subset_name} Set ]", attrs=["bold"]))

            print("Confusion matrix:")
            print(confusion_matrix(labels_eval, preds))
            print()

            print("Classification report:")
            print(classification_report(labels_eval, preds))

With the training and evaluation routine implemented, let's first run it on the **Byte-Histogram and Endianness Features**:


In [15]:
train_test_ml_models(feats_byte_hist_endian, labels)

[[ Training and Evaluation for SVM ]]
[ Training ]
Timing started for training of SVM ...
Timing ended for training of SVM. Elapsed time: 0.40s
[ Training Set ]
Confusion matrix:
[[804   0   0   3   1   0   4   2   1   2   0   3]
 [  0 781   1   0   0   1   0   0   1   0   0   0]
 [  3   3 801   0   2   0   3   6   1   1   1   1]
 [  0   0   0 821   1   0   2   5   1   1   0   1]
 [  1   0   0   0 758   1   1   3   0   1   1   2]
 [  0   0   0   0   0 804   2   1   0   0   0   0]
 [  2   2   1   1   0   1 811   6   0   1   1   3]
 [  1   0   1   0   0   0   4 822   0   0   0   0]
 [  0   0   2   0   3   1   0   5 513 306   2   1]
 [  0   0   1   2   5   0   3   3 311 502   1   0]
 [  1   0   0   0   0   0   1   1   0   0 829   0]
 [  1   4   1   0   0   1   1   5   0   0   0 828]]

Classification report:
              precision    recall  f1-score   support

           0       0.99      0.98      0.98       820
           1       0.99      1.00      0.99       784
           2       0.

... and for **Byte-level 1,2,3-Gram TF-IDF Features**:

In [16]:
train_test_ml_models(feats_tf_idf_123, labels)

[[ Training and Evaluation for SVM ]]
[ Training ]
Timing started for training of SVM ...
Timing ended for training of SVM. Elapsed time: 0.57s
[ Training Set ]
Confusion matrix:
[[820   0   0   0   0   0   0   0   0   0   0   0]
 [  0 784   0   0   0   0   0   0   0   0   0   0]
 [  0   0 822   0   0   0   0   0   0   0   0   0]
 [  0   0   0 832   0   0   0   0   0   0   0   0]
 [  0   0   0   0 768   0   0   0   0   0   0   0]
 [  0   0   0   0   0 807   0   0   0   0   0   0]
 [  0   0   0   0   0   0 829   0   0   0   0   0]
 [  0   0   0   0   0   0   0 828   0   0   0   0]
 [  0   0   0   0   0   0   0   0 833   0   0   0]
 [  0   0   0   0   0   0   0   0   0 828   0   0]
 [  0   0   0   0   0   0   0   0   0   0 832   0]
 [  0   0   0   0   0   0   0   0   0   0   0 841]]

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       820
           1       1.00      1.00      1.00       784
           2       1.

Let's check the shape of the byte-level TF-IDF feature matrix before deciding on dimensionality reduction:


In [17]:
feats_tf_idf_123.shape

(49121, 46638)

We'll find out that for **Byte-level 1,2,3-Gram TF-IDF Features**, there are more than 32,000 feature dimensions per sample. To ease the computational burden, we apply `KernelPCA` with an RBF kernel to reduce dimensionality to 300 components. Because `KernelPCA` does not natively support sparse input, we first fit it on a randomly-selected 20% subset of the data to reduce memory usage, then transform the full dataset.


In [18]:
from sklearn.decomposition import KernelPCA

KERNEL_PCA_DIMS = 300

# KernelPCA is fitted on a 20% stratified subset to reduce memory and runtime.
# The full dataset is then transformed using the fitted model.
feats_tf_idf_123_kpca_train, _ = train_test_split(
    feats_tf_idf_123, train_size=0.2, stratify=labels, random_state=RNG_SEED
)

with timeit("Kernel PCA fitting"):
    kpca_tf_idf_123 = KernelPCA(n_components=KERNEL_PCA_DIMS, kernel="rbf", random_state=RNG_SEED)
    kpca_tf_idf_123.fit(feats_tf_idf_123_kpca_train)

with timeit("Kernel PCA dimensionality reduction"):
    feats_tf_idf_123_kpca = kpca_tf_idf_123.transform(feats_tf_idf_123)

Timing started for Kernel PCA fitting ...
Timing ended for Kernel PCA fitting. Elapsed time: 76.08s
Timing started for Kernel PCA dimensionality reduction ...
Timing ended for Kernel PCA dimensionality reduction. Elapsed time: 20.02s


We can now run training and evaluation using the dimensionality-reduced (300-component) byte-level TF-IDF features:


In [21]:
train_test_ml_models(feats_tf_idf_123_kpca, labels)

[[ Training and Evaluation for SVM ]]
[ Training ]
Timing started for training of SVM ...
Timing ended for training of SVM. Elapsed time: 1.65s
[ Training Set ]
Confusion matrix:
[[750   0   1   5   0   0   1  62   0   1   0   0]
 [  3 704   2  74   0   0   0   1   0   0   0   0]
 [ 23   2 764  13   0   0   1  15   0   2   1   1]
 [  0   0   0 816   0   0   0  14   0   1   0   1]
 [  7   0   0  66 474   0   0 118   6  20   5  72]
 [  0   0   0   6   0 724   0  76   0   0   0   1]
 [  3   0   0  17   0   0 790  19   0   0   0   0]
 [  0   0   0   6   0   0   0 822   0   0   0   0]
 [  0   0   0   7   0   0   0  17 805   1   1   2]
 [  0   0   0   6   0   0   0  13   6 801   2   0]
 [  0   0   0   1   0   0   0   7   0   0 824   0]
 [  0   0   0   5   0   0   0   8   0   0   0 828]]

Classification report:
              precision    recall  f1-score   support

           0       0.95      0.91      0.93       820
           1       1.00      0.90      0.94       784
           2       1.

/opt/anaconda3/envs/cyber_env/lib/python3.14/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/cyber_env/lib/python3.14/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/cyber_env/lib/python3.14/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is

Timing ended for training of Decision Tree. Elapsed time: 3.40s
[ Training Set ]
Confusion matrix:
[[820   0   0   0   0   0   0   0   0   0   0   0]
 [  0 784   0   0   0   0   0   0   0   0   0   0]
 [  0   0 822   0   0   0   0   0   0   0   0   0]
 [  0   0   0 832   0   0   0   0   0   0   0   0]
 [  0   0   0   0 768   0   0   0   0   0   0   0]
 [  0   0   0   0   0 807   0   0   0   0   0   0]
 [  0   0   0   0   0   0 829   0   0   0   0   0]
 [  0   0   0   0   0   0   0 828   0   0   0   0]
 [  0   0   0   0   0   0   0   0 833   0   0   0]
 [  0   0   0   0   0   0   0   0   0 828   0   0]
 [  0   0   0   0   0   0   0   0   0   0 832   0]
 [  0   0   0   0   0   0   0   0   0   0   0 841]]

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       820
           1       1.00      1.00      1.00       784
           2       1.00      1.00      1.00       822
           3       1.00      1.00      1.00    

Finally, let's run training and evaluation on the **Hex-level 1,2,3-Gram TF-IDF Features**:


In [22]:
train_test_ml_models(feats_tf_idf_hex_123, labels)

[[ Training and Evaluation for SVM ]]
[ Training ]
Timing started for training of SVM ...
Timing ended for training of SVM. Elapsed time: 1.40s
[ Training Set ]
Confusion matrix:
[[819   0   0   0   0   0   0   1   0   0   0   0]
 [  0 784   0   0   0   0   0   0   0   0   0   0]
 [  0   0 821   0   0   0   0   1   0   0   0   0]
 [  0   0   0 831   0   0   0   1   0   0   0   0]
 [  0   0   0   0 768   0   0   0   0   0   0   0]
 [  0   0   0   0   0 807   0   0   0   0   0   0]
 [  0   0   0   0   0   0 829   0   0   0   0   0]
 [  0   0   0   0   0   0   0 828   0   0   0   0]
 [  0   0   0   0   0   0   0   1 832   0   0   0]
 [  0   0   0   0   0   0   0   0   0 828   0   0]
 [  0   0   0   0   0   0   0   0   0   0 832   0]
 [  0   0   0   0   0   0   0   1   0   0   0 840]]

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       820
           1       1.00      1.00      1.00       784
           2       1.

## References

1. Tech challenge: Machine learning binaries,” Feb 2021. [Online]. Available: https://www.praetorian.com/challenges/machine-learning-challenge/#how-to-play
2. J. Clemens, “Automatic classification of object code using machine learning,” Digital Investigation, vol. 14, pp. S156–S162, 2015.
3. D. Sahabandu, J. S. Mertoguno and R. Poovendran, "A Natural Language Processing Approach for Instruction Set Architecture Identification," in IEEE Transactions on Information Forensics and Security, vol. 18, pp. 4086-4099, 2023, doi: 10.1109/TIFS.2023.3288456.

